In [ ]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [ ]:
%cd slt

/content/slt


In [ ]:
!chmod +x data/download.sh
!data/download.sh

--2021-01-03 20:38:58--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  20.8MB/s    in 1m 42s  

2021-01-03 20:40:41 (18.0 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-03 20:40:41--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  19.9MB/s    in 7.5s    

2021-01-03 20:40:48 (16.6 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [ ]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 13.1MB/s 
     |████████████████████████████████| 112kB 22.3MB/s 
     |████████████████████████████████| 163kB 30.2MB/s 
     |████████████████████████████████| 399kB 30.6MB/s 
     |████████████████████████████████| 2.3MB 31.9MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 2.4MB 59.5MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 296kB 62.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 13.1MB 51.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 20.2MB 159kB/s 
     |

In [ ]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!cp -r ../drive/MyDrive/slt/keypoints/dev_features.zip data/dev_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/train_features.zip data/train_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/test_features.zip data/test_features.zip

!unzip -q data/dev_features.zip -d data
!unzip -q data/train_features.zip -d data
!unzip -q data/test_features.zip -d data

In [ ]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['train', 'dev', 'test']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)
  for i, src in tqdm(enumerate(src_features)):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)
  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

7096it [00:33, 212.23it/s]
519it [00:06, 79.11it/s]
642it [00:07, 80.54it/s]


In [ ]:
!ls data/PHOENIX2014T

phoenix14t.pami0.dev   phoenix14t.pami0.train  phoenix14t.pami1.test
phoenix14t.pami0.test  phoenix14t.pami1.dev    phoenix14t.pami1.train


In [ ]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]
list_doc['training']['recognition_loss_weight'] = 40

In [ ]:
with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [15]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-03 21:27:17,569 	Gloss Alignment :	                                         D      D      
2021-01-03 21:27:17,569 	--------------------------------------------------------------------------------------------------------------------
2021-01-03 21:27:17,570 	Text Reference  :	und nun die wettervorhersage für morgen samstag den zweiten oktober .
2021-01-03 21:27:17,570 	Text Hypothesis :	und nun die wettervorhersage für morgen samstag den dritten oktober .
2021-01-03 21:27:17,570 	Text Alignment  :	                                                    S                
2021-01-03 21:27:17,570 ========================================================================================================================
2021-01-03 21:27:17,570 Logging Sequence: dev/20December_2010_Monday_tagesschau-3208
2021-01-03 21:27:17,570 	Gloss Reference :	NEUN GRAD REGION MINUS ACHT NORDOST
2021-01-03 21:27:17,570 	Gloss Hypothesis:	NEUN GRAD REGION 